<a href="https://colab.research.google.com/github/AlexGCas/permutation_entropy/blob/main/entropia_permutacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Primer punto, entropía de permutación

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
#manejo de elemeotos del sistema, como carpetas
import scipy.io as sio;
import numpy as np;
import matplotlib.pyplot as plt
import glob
import pandas as pd
import re

In [ ]:
s = np.array([4, 7, 9, 10, 6, 11, 3])

In [ ]:
def OverlappingMatixUnidimensionalArray(array, D = 3):
  '''returns overlapping matrix for an unidimensional array
  Tao: embedding time delay.
  D: number of elements of a column.
  m = 7 - (3 - 1)*1 = 5
  using a fixed value of Tao = 1
  '''
  Tao = 1
  n = int(np.ceil(len(s) / D))
  m = len(array) - (D-1)*Tao
  overlappingMatrix = np.zeros([n, m])
  '''llenar una matriz no es una operacion vectorial y por lo tanto es necesario
  realizarun for para llenar cada una de las posiciones.'''
  for i in range(D):
    '''llenar cada una de las filas'''
    overlappingMatrix[i, :] = array[i: m +i]
  return overlappingMatrix

In [ ]:
def permutationColumn(column):
  permutation_column = np.zeros(column.shape)
  array_ordenado = np.sort(column)
  for i in range(len(column)):
    '''for para llenar la columna de permutacion'''
    for j in range(len(column)):
      '''for para recorrer el array ordenado'''
      if column[i] == array_ordenado[j]:
        permutation_column[i] = j
        array_ordenado[j] = None
  return permutation_column

In [ ]:
def permutationMatrix(matrix):
  permutation_matrix = np.zeros(matrix.shape)
  for column in range(matrix.shape[1]):
    permutation_matrix[:, column] = permutationColumn(matrix[:, column])
  return permutation_matrix

In [ ]:
def compararArray(array1, array2):
  comparacion = False
  if (array1 == array2).all():
    comparacion = True
  return comparacion

In [ ]:
def contarVecesArray(array_simple, array_complejo):
  numero_veces = 0
  for array in array_complejo:
    if (compararArray(array_simple, array)):
      numero_veces = numero_veces + 1
  return numero_veces

In [ ]:
def eliminarPermutacion(array_simple, array_complejo):
  contador = 0
  array_modificado = []
  for array in array_complejo:
    if compararArray(array_simple, array):
      pass
    else:
      array_modificado.append(array)
  return array_modificado

In [ ]:
def countFrecuency(permutation_matrix, length = 7):
  numero_permutaciones = int(math.factorial(length)/math.factorial(permutation_matrix.shape[0]))
  freq_dist = {}
  permutaciones = []
  contador = 0
  for column in range(permutation_matrix.shape[1]):
    permutaciones.append(permutation_matrix[:, column])
  for column in range(permutation_matrix.shape[1]):
    veces = contarVecesArray(permutation_matrix[:, column], permutaciones)
    permutaciones = eliminarPermutacion(permutation_matrix[:, column], permutaciones)
    if veces == 0:
      pass
    else:
      freq_dist[contador] = veces
      contador = contador + 1
  return freq_dist


In [ ]:
def permutationEntropy(freq_dict):
  frecuencias = np.array(
      list(
          freq_dict.values() 
          ))
  probabilidades = frecuencias / np.sum(frecuencias)
  logaritmos = np.log2(probabilidades)
  PE = -np.dot(logaritmos, probabilidades)
  return PE

In [ ]:
def permutationEntropyFromData(array):
  overlappingMatrix = OverlappingMatixUnidimensionalArray(array)
  permutation_matrix = permutationMatrix(overlappingMatrix)
  freq_dist = countFrecuency(permutation_matrix)
  permutation_entropy = permutationEntropy(freq_dist)
  return permutation_entropy

In [ ]:
def extraccionNombre(archivo):
  nombre = re.findall("([A-Za-z0-9_]+).mat", archivo)
  return nombre[0]
def extraccionTipo(archivo):
  tipo = re.findall("([a-z]+):?/[A-Za-z0-9_]+.mat", archivo)
  return tipo[0]

In [ ]:
permutationEntropyFromData(s)

1.5219280948873621

In [ ]:
archivos_g1 = glob.glob('/content/drive/Shareddrives/biosenales_sistemas/datasets_proyecto1/control/*.mat')
archivos_g2 = glob.glob('/content/drive/Shareddrives/biosenales_sistemas/datasets_proyecto1/parkinson/*.mat')

In [ ]:
#para un canal
data = sio.loadmat(archivos_g1[1])
senal = data['data']
epoca = senal[0, :, 0]
entropia_canal = permutationEntropyFromData(epoca)
entropia_canal

1.2755402265848075


# Punto 2

In [ ]:
def prom_entropy_epocas(x): #entrada de x con 176 epocas 
  p = np.zeros(x.shape[1]);
  for i in range (x.shape[1]):
    p[i] = permutationEntropyFromData(x[:,i])
  prom = np.mean(p)
  return prom

In [ ]:
promedio_epocas = prom_entropy_epocas(senal[0,:,:])
promedio_epocas

1.2562825119080925

# Punto 3

In [ ]:
def prom_entropy_canales(x):
  c = np.zeros(x.shape[0]);
  for i in range (x.shape[0]):
    c[i] = prom_entropy_epocas(x[i,:,:])
  return c

In [ ]:
entropia_canales = prom_entropy_canales(senal)

In [ ]:
entropia_canales

array([1.25628251, 1.25798283, 1.26337211, 1.26128997, 1.25968977,
       1.23764416, 1.24285515, 1.24403065])

# Punto 4

# Nueva sección

In [ ]:
def base_datos(archivos):
  tabla = pd.DataFrame({},index=[0])
  pacientes = {}
  for i in range (len(archivos)):
    data = sio.loadmat(archivos[i]);
    senal = data['data'];
    nombre = extraccionNombre(archivos[i])
    estado = extraccionTipo(archivos[i])
    paciente = prom_entropy_canales(senal);
    pacientes['nombre']= nombre;
    pacientes['estado']= estado;
    pacientes['canal0'] = paciente[0];
    pacientes['canal1'] = paciente[1];
    pacientes['canal2'] = paciente[2];
    pacientes['canal3'] = paciente[3];
    pacientes['canal4'] = paciente[4];
    pacientes['canal5'] = paciente[5];
    pacientes['canal6'] = paciente[6];
    pacientes['canal7'] = paciente[7];
    # df3 = pd.DataFrame(pacientes,index=[0])
    tabla = tabla.append(pacientes, ignore_index = True)
  tabla = tabla.drop([0])
  return tabla

In [ ]:
control = base_datos(archivos_g1)
control.to_csv("control.csv")

In [ ]:
parkinson = base_datos(archivos_g2)
parkinson.to_csv("parkinson.csv")